In [3]:
# ruff: noqa: RUF001

import os

import basedosdados as bd
import pandas as pd

INPUT = os.path.join("models", "br_inep_educacao_especial", "data")
OUTPUT = os.path.join("models", "br_inep_educacao_especial", "output")

os.makedirs(INPUT, exist_ok=True)
os.makedirs(OUTPUT, exist_ok=True)

/home/laribrito/BD/pipelines/.venv/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
def read_sheet(sheet_name: str, skiprows: int = 3) -> pd.DataFrame:
    return pd.read_excel(
        os.path.join(INPUT, "TDI_ANO_2020_21_22_23_24.xlsx"),
        skiprows=skiprows,
        sheet_name=sheet_name,
    )

In [5]:
# Load the Excel file into a pandas ExcelFile object
excel_data = pd.ExcelFile(os.path.join(INPUT, "TDI_ANO_2020_21_22_23_24.xlsx"))

# Get the sheet names
print(excel_data.sheet_names)

['TDI_ANO_2020_21_22_23_24_educac']


In [6]:
df = excel_data.parse()

In [7]:
# Print the column names of the DataFrame to see what was read from the Excel sheet
print(df.columns)

Index(['NU_ANO_CENSO', 'TP_TIPO_CLASSE', 'CO_REGIAO', 'NO_REGIAO', 'CO_UF',
       'SG_UF', 'TP_DEPENDENCIA', 'NO_DEPENDENCIA', 'NO_CATEGORIA',
       'FUN_CAT_0', 'FUN_AI_CAT_0', 'FUN_AF_CAT_0', 'FUN_01_CAT_0',
       'FUN_02_CAT_0', 'FUN_03_CAT_0', 'FUN_04_CAT_0', 'FUN_05_CAT_0',
       'FUN_06_CAT_0', 'FUN_07_CAT_0', 'FUN_08_CAT_0', 'FUN_09_CAT_0',
       'MED_CAT_0', 'MED_01_CAT_0', 'MED_02_CAT_0', 'MED_03_CAT_0',
       'MED_04_CAT_0'],
      dtype='object')


In [8]:
# -----------------------------
# Rename and filter columns
# -----------------------------
# This block renames the DataFrame columns according to the RENAME_COLUMNS dictionary
# and keeps only the renamed columns. It overwrites the original df variable, so
# df will contain only the columns specified in RENAME_COLUMNS.

RENAME_COLUMNS = {
    "NU_ANO_CENSO": "ano",
    "NO_CATEGORIA": "categoria",
    "NO_REGIAO": "regiao",
    "SG_UF": "sigla_uf",
    "TP_TIPO_CLASSE": "classe",
    "NO_DEPENDENCIA": "dependencia",
    "FUN_AI_CAT_0": "Ensino Fundamental – Anos Iniciais",
    "FUN_AF_CAT_0": "Ensino Fundamental – Anos Finais",
    "MED_CAT_0": "Ensino Médio Regular",
}


def keep_only_renamed(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns=RENAME_COLUMNS)

    cols_keep = list(RENAME_COLUMNS.values())

    cols_existentes = [col for col in cols_keep if col in df.columns]

    return df[cols_existentes]


df = keep_only_renamed(df)
print(df.columns)

Index(['ano', 'categoria', 'regiao', 'sigla_uf', 'classe', 'dependencia',
       'Ensino Fundamental – Anos Iniciais',
       'Ensino Fundamental – Anos Finais', 'Ensino Médio Regular'],
      dtype='object')


In [9]:
# Filters only years equal to or greater than 2022
df = df[
    (df["ano"] >= 2022)
    & (df["classe"] != "0 - Todas as turmas")
    & (df["categoria"] == "Modalidade: educação especial")
    & (df["dependencia"] == "Total")
    & (df["sigla_uf"].notna())
]
df

,ano,categoria,regiao,sigla_uf,classe,dependencia,Ensino Fundamental – Anos Iniciais,Ensino Fundamental – Anos Finais,Ensino Médio Regular
2043,2022,Modalidade: educação especial,Norte,RO,1 - Classe comum,Total,14.4,37.1,43.0
2055,2022,Modalidade: educação especial,Norte,AC,1 - Classe comum,Total,22.5,43.3,44.4
2067,2022,Modalidade: educação especial,Norte,AM,1 - Classe comum,Total,27.0,51.7,52.4
2079,2022,Modalidade: educação especial,Norte,RR,1 - Classe comum,Total,17.2,33.5,38.0
2091,2022,Modalidade: educação especial,Norte,PA,1 - Classe comum,Total,35.3,65.3,63.9
...,...,...,...,...,...,...,...,...,...
3883,2024,Modalidade: educação especial,Sul,RS,1 - Classe comum,Total,16.0,41.0,46.3
3895,2024,Modalidade: educação especial,Centro-Oeste,MS,1 - Classe comum,Total,26.2,47.7,49.9
3907,2024,Modalidade: educação especial,Centro-Oeste,MT,1 - Classe comum,Total,6.1,22.8,30.6
3919,2024,Modalidade: educação especial,Centro-Oeste,GO,1 - Classe comum,Total,12.1,26.1,31.3


In [10]:
# Filters the DataFrame to keep only rows where 'regiao' is "Brasil"
# and melts the DataFrame from wide to long format (one row per metric)
# Each row will have: 'ano', 'regiao', 'metrica' (original metric name), and 'valor' (corresponding value)
melted_dataframe = pd.concat(
    [
        df.pipe(lambda d: d.loc[(d["sigla_uf"] != " ")]).pipe(
            lambda d: pd.melt(
                d,
                id_vars=["ano", "sigla_uf"],
                value_vars=d.columns.difference(
                    ["ano", "sigla_uf"]
                ).tolist(),  # Convert to list
                var_name="metrica",
                value_name="tdi",
            )
        )
    ]
)

In [11]:
melted_dataframe

,ano,sigla_uf,metrica,tdi
0,2022,RO,Ensino Fundamental – Anos Finais,37.1
1,2022,AC,Ensino Fundamental – Anos Finais,43.3
2,2022,AM,Ensino Fundamental – Anos Finais,51.7
3,2022,RR,Ensino Fundamental – Anos Finais,33.5
4,2022,PA,Ensino Fundamental – Anos Finais,65.3
...,...,...,...,...
562,2024,RS,regiao,Sul
563,2024,MS,regiao,Centro-Oeste
564,2024,MT,regiao,Centro-Oeste
565,2024,GO,regiao,Centro-Oeste


In [12]:
melted_dataframe["sigla_uf"].unique()

array(['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE', 'RN',
       'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR', 'SC',
       'RS', 'MS', 'MT', 'GO', 'DF'], dtype=object)

In [13]:
melted_dataframe["etapa_ensino"] = melted_dataframe["metrica"].apply(
    lambda v: v.split("_")[-1]
)  # Extracts 'anosiniciais', 'anosfinais', or 'ensinomedio'
melted_dataframe["tipo_metrica"] = melted_dataframe["metrica"].apply(
    lambda v: v.split("_")[0]
)  # Extracts 'tdi'
melted_dataframe["tdi"] = pd.to_numeric(
    melted_dataframe["tdi"], errors="coerce"
)

# Pivoting the melted DataFrame to get desired columns
df_final = melted_dataframe.pivot_table(
    index=["ano", "sigla_uf", "etapa_ensino"],
    columns="tipo_metrica",
    values="tdi",
).reset_index()

In [14]:
melted_dataframe

,ano,sigla_uf,metrica,tdi,etapa_ensino,tipo_metrica
0,2022,RO,Ensino Fundamental – Anos Finais,37.1,Ensino Fundamental – Anos Finais,Ensino Fundamental – Anos Finais
1,2022,AC,Ensino Fundamental – Anos Finais,43.3,Ensino Fundamental – Anos Finais,Ensino Fundamental – Anos Finais
2,2022,AM,Ensino Fundamental – Anos Finais,51.7,Ensino Fundamental – Anos Finais,Ensino Fundamental – Anos Finais
3,2022,RR,Ensino Fundamental – Anos Finais,33.5,Ensino Fundamental – Anos Finais,Ensino Fundamental – Anos Finais
4,2022,PA,Ensino Fundamental – Anos Finais,65.3,Ensino Fundamental – Anos Finais,Ensino Fundamental – Anos Finais
...,...,...,...,...,...,...
562,2024,RS,regiao,NaN,regiao,regiao
563,2024,MS,regiao,NaN,regiao,regiao
564,2024,MT,regiao,NaN,regiao,regiao
565,2024,GO,regiao,NaN,regiao,regiao


In [ ]:
# Remove all rows where the column 'valor' has missing (NaN) values.
melted_dataframe = melted_dataframe.dropna(subset=["tdi"])

In [ ]:
# Select and keep only the specified columns from melted_dataframe
# This ensures the DataFrame contains only the relevant variables for analysis
melted_dataframe = melted_dataframe[
    [
        "ano",
        "sigla_uf",
        "etapa_ensino",
        "tdi",
    ]
]

In [17]:
melted_dataframe

,ano,sigla_uf,etapa_ensino,tdi
0,2022,RO,Ensino Fundamental – Anos Finais,37.1
1,2022,AC,Ensino Fundamental – Anos Finais,43.3
2,2022,AM,Ensino Fundamental – Anos Finais,51.7
3,2022,RR,Ensino Fundamental – Anos Finais,33.5
4,2022,PA,Ensino Fundamental – Anos Finais,65.3
...,...,...,...,...
238,2024,RS,Ensino Médio Regular,46.3
239,2024,MS,Ensino Médio Regular,49.9
240,2024,MT,Ensino Médio Regular,30.6
241,2024,GO,Ensino Médio Regular,31.3


In [ ]:
# Define the output file path by joining the OUTPUT directory with a subfolder name
path = os.path.join(OUTPUT, "educacao_especial_brasil_distorcao_idade_serie")
# Create the directory if it does not already exist
os.makedirs(path, exist_ok=True)
# Convert all values in df_final to string (astype(str)),
# then save it as a CSV file inside the specified folder.
melted_dataframe.astype(str).to_csv(
    os.path.join(path, "sigla_uf_tdi_2022_2024.csv"), index=False
)

In [ ]:
# Read a table directly from BigQuery into a pandas DataFrame using the basedosdados library.
# The SQL query selects all columns from the table:
#   basedosdados.br_inep_educacao_especial.uf_taxa_rendimento
# The parameter billing_project_id specifies which GCP project will be billed for the query.
df_bq = bd.read_sql(
    "select * from basedosdados.br_inep_educacao_especial.uf_distorcao_idade_serie",
    billing_project_id="basedosdados-dev",
)

Downloading: 100%|██████████|


In [20]:
df_bq

,ano,sigla_uf,etapa_ensino,tdi
0,2007,PI,Ensino Médio Regular,80.5
1,2007,PR,Ensino Médio Regular,59.5
2,2007,MT,Ensino Médio Regular,78.1
3,2007,DF,Ensino Médio Regular,69.6
4,2007,PA,Ensino Médio Regular,77.7
...,...,...,...,...
1210,2021,AM,Ensino Fundamental – Anos Iniciais,39.8
1211,2021,RN,Ensino Fundamental – Anos Iniciais,23.6
1212,2021,MG,Ensino Fundamental – Anos Iniciais,25.0
1213,2021,SC,Ensino Fundamental – Anos Iniciais,14.6


In [ ]:
# Concatenate two DataFrames.
df_updated = pd.concat([df_bq, melted_dataframe])

In [ ]:
# Convert all values in df_updated to strings and save as a CSV file.
df_updated.astype(str).to_csv(
    os.path.join(path, "uf_distorcao_idade_serie.csv"), index=False
)

In [ ]:
# Create a Table object representing a BigQuery table in the specified dataset.
tb_uf = bd.Table(
    dataset_id="br_inep_educacao_especial", table_id="uf_distorcao_idade_serie"
)
# Upload the local CSV file to the BigQuery table.
# Parameters:
# - if_storage_data_exists='replace': replace the data in storage if it already exists
# - if_table_exists='replace': replace the table if it already exists
# - source_format='csv': specify that the source file is a CSV
tb_uf.create(
    os.path.join(path, "uf_distorcao_idade_serie.csv"),
    if_storage_data_exists="replace",
    if_table_exists="replace",
    source_format="csv",
)

Uploading files: 100%|██████████| 1/1 [00:01<00:00,  1.96s/it]
2025-08-26 20:44:33.920 | WARNING  | basedosdados.upload.table:_load_staging_schema_from_data:105 -  Table uf_distorcao_idade_serie allready exists, replacing schema!
2025-08-26 20:44:35.343 | INFO     | basedosdados.upload.table:delete:809 -  Table uf_distorcao_idade_serie_staging was deleted!
2025-08-26 20:44:35.695 | SUCCESS  | basedosdados.upload.table:create:690 - Table uf_distorcao_idade_serie was created in staging!
